# Flipkart EDA assignment

## TODO: Upload csv file before moving to next

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Flipkart Data Engineering").getOrCreate()
file_path = '/FileStore/tables/Flipkart.csv'
flipkart_df = spark.read.csv(file_path, header=True, inferSchema=True)

In [0]:
flipkart_df.printSchema()
flipkart_df.count()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- maincateg: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- actprice1: integer (nullable = true)
 |-- norating1: integer (nullable = true)
 |-- noreviews1: integer (nullable = true)
 |-- star_5f: integer (nullable = true)
 |-- star_4f: integer (nullable = true)
 |-- star_3f: integer (nullable = true)
 |-- star_2f: integer (nullable = true)
 |-- star_1f: integer (nullable = true)
 |-- fulfilled1: integer (nullable = true)

Out[5]: 5244

In [0]:
flipkart_df.show(5)

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|      999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|         1|
|20532|Women Black Flats...|   3.9|    Women|Flipkart|      499|     3015|       404|   1458|    657|    397|    182|    321|         1|
|10648|Women Gold Wedges...|   3.9|    Women|Flipkart|      999|      449|        52|    229|     70|     71|     33|     46|         1|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart|     2999|      290|        40|    141|     51|     49|     17|     32|         1|
|12593|Loafers For Men  ...|   3.9|      

In [0]:
# Display all category names

flipkart_df.select("maincateg").distinct().show(truncate=False)

+---------+
|maincateg|
+---------+
|null     |
|Men      |
|Women    |
+---------+



In [0]:
# Filter products with rating > 4 and more than 100 reviews

# Convert 'Rating' and 'noreviews1' columns to numeric types just in case they're read as strings
from pyspark.sql.functions import col

filtered_df = flipkart_df \
    .filter((col("Rating").cast("double") > 4) & (col("noreviews1").cast("int") > 100))

filtered_df.show(truncate=False)


+-----+--------------------------------------------------------------------------------------------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|id   |title                                                                                                   |Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------------------------------------------------------------------------------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|13859|Women Black Wedges Sandal                                                                               |4.2   |Women    |Flipkart|279      |3048     |487       |1746   |679    |348    |114    |161    |1         |
|19453|Denill Ankle Length Sneakers Sneakers For Women  (Pink)                                                 |4.1 

In [0]:
# Display Products in 'Men' category that are fulfilled

from pyspark.sql.functions import col

men_fulfilled_df = flipkart_df \
    .filter((col("maincateg") == "Men") & (col("fulfilled1") == "1"))

men_fulfilled_df.show(truncate=False)


+-----+----------------------------------------------------------------------------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|id   |title                                                                                   |Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+----------------------------------------------------------------------------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|2242 |Casuals For Men  (Blue)                                                                 |3.8   |Men      |Flipkart|999      |27928    |3543      |14238  |4295   |3457   |1962   |3976   |1         |
|20677|Men's Height Increasing High Heel Formal Party Wear Slip-on Boots Slip On For Men  (Tan)|3.9   |Men      |Flipkart|2999     |290      |40        |141    |51     |49     |17 

In [0]:
# Dsiplay number of products per category

flipkart_df.groupBy("maincateg").count().show()


+---------+-----+
|maincateg|count|
+---------+-----+
|     null|   67|
|      Men| 2145|
|    Women| 3032|
+---------+-----+



In [0]:
# Display Average rating per category

from pyspark.sql.functions import col, avg

# Make sure Rating is treated as numeric
avg_rating_df = flipkart_df \
    .withColumn("Rating", col("Rating").cast("double")) \
    .groupBy("maincateg") \
    .agg(avg("Rating").alias("avg_rating"))

# Show the result sorted by average rating (optional)
avg_rating_df.orderBy("avg_rating", ascending=False).show(truncate=False)


+---------+------------------+
|maincateg|avg_rating        |
+---------+------------------+
|null     |4.225             |
|Women    |4.0416582064297915|
|Men      |3.9662970106075246|
+---------+------------------+



In [0]:
# Dsiplay Category with highest average number of reviews

from pyspark.sql.functions import col, avg

# Ensure 'noreviews1' is numeric and calculate average reviews per category
avg_reviews_df = flipkart_df \
    .withColumn("noreviews1", col("noreviews1").cast("int")) \
    .groupBy("maincateg") \
    .agg(avg("noreviews1").alias("avg_reviews"))

# Get the category with the highest average number of reviews
avg_reviews_df.orderBy("avg_reviews", ascending=False).show(1, truncate=False)


+---------+-----------------+
|maincateg|avg_reviews      |
+---------+-----------------+
|Men      |759.6554778554779|
+---------+-----------------+
only showing top 1 row



In [0]:
# Top 5 products with highest price. display product name and price

from pyspark.sql.functions import col

# Cast actprice1 to numeric and select top 5 by price
top_price_df = flipkart_df \
    .withColumn("actprice1", col("actprice1").cast("int")) \
    .select("title", "actprice1") \
    .orderBy(col("actprice1").desc()) \
    .limit(5)

top_price_df.show(truncate=False)


+----------------------------------------------------------+---------+
|title                                                     |actprice1|
+----------------------------------------------------------+---------+
|GEL-QUANTUM 360 KNIT Running Shoes For Women  (Blue, Grey)|15999    |
|Women Orange Sports Sandal                                |12999    |
|GT-2000 5 (D) Running Shoes For Women  (Blue)             |10999    |
|Hybrid Astro Running Shoes For Men  (Black)               |10999    |
|DEERUPT RUNNER W Sneakers For Women  (White)              |9999     |
+----------------------------------------------------------+---------+



In [0]:
# Display Min, max, and avg price per category

from pyspark.sql.functions import col, min, max, avg

flipkart_df.groupBy("maincateg").agg({"actprice1": "min", "actprice1": "max", "actprice1": "avg"}).show()


  File <command-4003054415232855>:5
    flipkart_df.groupBy("maincateg").agg({"actprice1": "min", "actprice1": "max", "actprice1": "avg"}).show()
                                                                                              ^
SyntaxError: invalid non-printable character U+00A0


In [0]:
# Display number of nulls in each column

from pyspark.sql.functions import col, sum, when

# Count nulls in each column
null_counts = flipkart_df.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in flipkart_df.columns
])

null_counts.show(truncate=False)


+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|id |title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|0  |0    |203   |67       |0       |0        |0        |0         |68     |0      |0      |0      |186    |0         |
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+



In [0]:
# calculate and display the category name, number of fulfilled, and unfulfilled products

from pyspark.sql.functions import col, sum, when

# Ensure fulfilled1 is treated consistently (e.g., as int or string depending on your dataset)
fulfilled_df = flipkart_df.groupBy("maincateg").agg(
    sum(when(col("fulfilled1") == "1", 1).otherwise(0)).alias("fulfilled_count"),
    sum(when(col("fulfilled1") != "1", 1).otherwise(0)).alias("unfulfilled_count")
)

fulfilled_df.orderBy("maincateg").show(truncate=False)


+---------+---------------+-----------------+
|maincateg|fulfilled_count|unfulfilled_count|
+---------+---------------+-----------------+
|null     |40             |27               |
|Men      |1352           |793              |
|Women    |1778           |1254             |
+---------+---------------+-----------------+



In [0]:
flipkart_df.groupBy("maincateg").count().show()

+---------+-----+
|maincateg|count|
+---------+-----+
|     null|   67|
|      Men| 2145|
|    Women| 3032|
+---------+-----+



In [0]:
from pyspark.sql.functions import avg

flipkart_df.groupBy("maincateg").agg(avg("Rating").alias("avg_rating")).show()

+---------+------------------+
|maincateg|        avg_rating|
+---------+------------------+
|     null|             4.225|
|      Men|3.9662970106075246|
|    Women|4.0416582064297915|
+---------+------------------+



In [0]:
# Display Category with highest average number of reviews

from pyspark.sql.functions import avg

flipkart_df.groupBy("maincateg").agg(avg("noreviews1").alias("avg_reviews")).orderBy("avg_reviews", ascending=False).show(1)

+---------+-----------------+
|maincateg|      avg_reviews|
+---------+-----------------+
|      Men|759.6554778554779|
+---------+-----------------+
only showing top 1 row



In [0]:
category_counts = flipkart_df.groupBy("maincateg").count()
category_counts.display()

maincateg,count
null,67
Men,2145
Women,3032


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import avg

category_avg_ratings = flipkart_df.groupBy("maincateg").agg(avg("Rating").alias("avg_rating"))
category_avg_ratings.display()

maincateg,avg_rating
null,4.225
Men,3.9662970106075246
Women,4.0416582064297915


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import sum

category_total_reviews = flipkart_df.groupBy("maincateg").agg(sum("noreviews1").alias("total_reviews"))
category_total_reviews.display()

maincateg,total_reviews
null,14084
Men,1629461
Women,535290


In [0]:
from pyspark.sql.functions import col, max

flipkart_df = flipkart_df.withColumn("star_5f", col("star_5f").cast("int"))
max_val = flipkart_df.agg(max("star_5f")).first()[0]
flipkart_df.filter(col("star_5f") == max_val).select("title", "star_5f").show(truncate=False)


+-------------------------------------------+-------+
|title                                      |star_5f|
+-------------------------------------------+-------+
|Jio-13 Running Shoes For Men  (Black, Grey)|151193 |
|Running Shoes For Men  (Silver, Black)     |151193 |
+-------------------------------------------+-------+

